In [ ]:
# commands used to pull down CSV:
# gtt report --report=records --output=csv --no_headlines -c --time_format="[%sign][%hours_overall]" msoe.edu/sdl/create-institute/backend --file="./gitlab-time-tracker-backend.csv"
# gtt report --report=records --output=csv --no_headlines -c --time_format="[%sign][%hours_overall]" msoe.edu/sdl/create-institute/frontend --file="./gitlab-time-tracker-frontend.csv"

import pandas as pd
import numpy as np
from math import isnan

# fix notebook styling
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

df_frontend = pd.read_csv('./gitlab-time-tracker-frontend.records.csv')
df_backend = pd.read_csv('./gitlab-time-tracker-backend.records.csv')
df_frontend['repo'] = 'frontend'
df_backend['repo'] = 'backend'
df = pd.concat([df_frontend, df_backend], sort=True)

NAME_MAP = {
    "stanglersm": "stangler",
    "brandonakandy": "kandarapally",
    "martensdr": "martens",
    "gnabasikat1": "gnabasik",
    "noe.gonzalez": "gonzalez",
    "gieseba": "giese",
    "slang": "lang",
}

def normalize_name(x):
    return NAME_MAP.get(x.user, x.user)

df['user'] = df.apply(normalize_name, axis=1)

def create_link(x):
    if x.type == "Issue":
        return f"https://gitlab.com/msoe.edu/sdl/create-institute/{x.repo}/issues/{x.iid}"
    elif x.type == "MergeRequest":
        return f"https://gitlab.com/msoe.edu/sdl/create-institute/{x.repo}/merge_requests/{x.iid}"
    else:
        return ""

format_hours = lambda x: '0' if x == 0 else "{0:.2f}".format(x).rstrip('0').rstrip('.') + 'h'

# deal with time corrections
df = df.groupby(['date', 'iid', 'repo', 'type', 'user'], as_index=False).agg('sum')
df = df[0 != df.time]

df['link'] = df.apply(create_link, axis=1)
df['date'] = pd.to_datetime(df['date'], dayfirst=True)

# drop this row
# 2019-01-07 00:00:00	24	frontend	Issue	kandarapally	1h	https://gitlab.com/msoe.edu/sdl/create-institute/frontend/issues/24
df = df[(df.user != 'kandarapally') | (df.date != '2019-01-07 00:00:00') | (df.link != 'https://gitlab.com/msoe.edu/sdl/create-institute/frontend/issues/24')]

df.sort_values(by=['date', 'user'], inplace=True, ascending=False)

df = df.reset_index(drop=True)

TOPIC_REPLACEMENT_MAP = {
    "backend !4": "admin auth",
    "backend !18": "make phone number optional",
    "backend !21": "remove static plug & websocket",
    "backend !22": "upgrade to phoenix 1.4",
    "backend !23": "remove unused dep",
    "backend !24": "move endpoint out of pipeline",
    "backend !25": "remove description required",
    "backend !27": "add pagination",
    "backend !29": "backend image order",
    "backend #21": "add pagination",
    "backend #23": "backend image order",
    "backend #6": "email admin on project creation",
    "backend #9": "admin auth",
    "frontend !15": "project gallery pictures admin",
    "frontend !26": "home page",
    "frontend !28": "project gallery pictures admin",
    "frontend !29": "fix edit page issues",
    "frontend !31": "change photos on index page",
    "frontend #8": "new account registration form",
    "frontend #9": "admin login form",
    "frontend #16": "project gallery pictures admin",
    "frontend #22": "admin edit form verification",
    "frontend #24": "Q2 Flex Week",
    "frontend #25": "status badges",
    "frontend #27": "project-form-confirm-submission",
    "frontend #31": "project gallery pictures admin",
    "frontend #33": "home page",
    "frontend #37": "Q2 sprint 2 overhead",
    "frontend #29": "project form styling",
    "frontend #36": "Q2 sprint 2 ceremonies",
    "frontend #43": "search bar",
    "frontend #51": "home page",
}

def create_topic(x):
    if x.type == "Issue":
        marker = f"{x.repo} #{x.iid}"
    elif x.type == "MergeRequest":
        marker = f"{x.repo} !{x.iid}"
    else:
        marker = ""

    return TOPIC_REPLACEMENT_MAP.get(marker, marker)
    

df['topic'] = df.apply(create_topic, axis=1)

pd.options.display.max_rows = None
pd.options.display.max_columns = None
pd.set_option('max_colwidth', 800)

df.style.format({
    'time': format_hours
})

In [ ]:
df['year'] = df['date'].dt.strftime('%Y')
df['month'] = df['date'].dt.strftime('%m')
df['day'] = df['date'].dt.strftime('%d (%a)')
df['week'] = df['date'].dt.strftime('%W')

res = df.pivot_table(
    index='user',
    columns=['year', 'week', 'month', 'day'],
    values='time',
    aggfunc=[np.sum],
    fill_value=0,
    margins=True,
    margins_name='Σ'
)
res.style.format(format_hours)

In [ ]:
res_weeks = df.pivot_table(
    index='user',
    columns=['week'],
    values='time',
    aggfunc=[np.sum],
    fill_value=0,
    margins=True,
    margins_name='Σ'
)
res_weeks.style.format(format_hours)

In [ ]:
# show number of days in each numbered week (sanity check)
res_weeks = df.groupby(['week'])['day'].nunique()
res_weeks

In [ ]:
import altair as alt

alt.renderers.enable('notebook')

chart_data = df.groupby(['user', 'date'], as_index=False).agg('sum')

alt.Chart(chart_data).mark_circle(
    opacity=0.8,
    stroke='black',
    strokeWidth=1
).encode(
    alt.X('date:O', axis=alt.Axis(labelAngle=0)),
    alt.Y('user:N'),
    alt.Size('time:Q',
        scale=alt.Scale(range=[0, 1000]),
        legend=alt.Legend(title='hours logged in day')
    ),
    alt.Color('user:N', legend=None)
).properties(
    width=900,
    height=500
)

In [ ]:
df['day_of_week'] = df['date'].dt.strftime('%a')
chart_data = df.groupby(['user', 'day_of_week'], as_index=False).agg('sum')

alt.Chart(chart_data).mark_circle(
    opacity=0.8,
    stroke='black',
    strokeWidth=1
).encode(
    alt.X('day_of_week:O', axis=alt.Axis(labelAngle=0), sort=['Mon','Tue','Wed','Thu','Fri','Sat', 'Sun']),
    alt.Y('user:N'),
    alt.Size('time:Q',
        scale=alt.Scale(range=[0, 5000]),
        legend=alt.Legend(title='hours logged in day of week')
    ),
    alt.Color('user:N', legend=None)
).properties(
    width=900,
    height=500
)

In [ ]:
# regroup because some PBI mappings group issues/MRs 
chart_data = df.groupby(['user', 'topic'], as_index=False).agg('sum')

base = alt.Chart(chart_data)
scale = alt.Scale(paddingInner=0)
heatmap = base.mark_rect().encode(
    alt.X('user:O', scale=scale),
    alt.Y('topic:O', scale=scale),
    color='time'
)

text = base.mark_text(baseline='middle').encode(
    x='user:O',
    y='topic:O',
    text='time',
    color=alt.condition(
        alt.datum['time'] > 14,
        alt.value('black'),
        alt.value('white')
    )
)

heatmap + text